[Reference](https://medium.com/analytics-vidhya/multi-tasking-your-way-in-python-795ced176d9d)

# A Client — Server Architecture

In [2]:
import os
import time
from time import perf_counter
from multiprocessing import Queue, Process, Event
import queue as q

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s : %(levelname)s : %(message)s')
logger = logging.getLogger(__name__)


class Worker(Process):
    
    def __init__(self, task_queue: Queue, 
                 stop_event: Event, **kwargs):
        super().__init__(**kwargs)
        self.task_queue = task_queue
        self.stop_event = stop_event
        self.name = kwargs['name']
    
    def run(self):
        logger.info(f'Intiailizing Worker - {self.name} ProcessID - {os.getpid()}')

        # While loop will run until both (Empty queue and stop event trigger by manager) is False      
        while not self.task_queue.empty() or not self.stop_event.is_set():

            try:
                job = self.task_queue.get_nowait() # This will get the jobs entered by manager into the queue
                
                # Perform operation on jobs
                logger.info(f'Starting process for JobID - {job} on worker - {self.name}')
                ####################################################################
                #                  Processing tasks starts 
                ####################################################################
                start_job = perf_counter()
                random_operation = sum([(job)^i for i in range(500000)])
                time.sleep(2)
                end_job = perf_counter()

                logger.info(f'Time taken to execute JobID - {job} on Worker - {self.name} is {end_job - start_job}')
                ####################################################################
                #                  Processing tasks ends 
                ####################################################################

            except q.Empty: pass
        logger.info(f"{self.name} - Process terminates")



class Manager:
    
    def __init__(self, n_workers: int = 1, 
                 max_tasks: int = 2000):
        self.stop_event = Event()
        self.task_queue = Queue(maxsize=max_tasks)

        n_workers = 1 if n_workers < 1 else n_workers
        logger.info(f"Starting {n_workers} workers in process mode")
        
        self.workers = [Worker(self.task_queue,
                            self.stop_event,
                            name=f"Worker{i}") 
                        for i in range(n_workers)]
        for worker in self.workers: worker.start()

    def add_jobs(self):
        '''Adds the jobs to the queue'''
        ####################################################################
        #                  Task addition on queue starts 
        ####################################################################

        for i in range(10):
            self.task_queue.put(i) # Assigns the job in queue
            time.sleep(0.1)

        ####################################################################
        #                 Task addition on queue ends 
        ####################################################################
        
    def terminate(self):
        '''Sets termiate event when called'''
        self.stop_event.set()
        
if __name__ == "__main__":

    n_workers =  os.cpu_count() - 1 # Gets no of cores present on the machine
    process = Manager(n_workers=n_workers) # Intialize Manager Object
    process.add_jobs() # Adds the jobs to process for workers in queue
    process.terminate() # Triggers termiante event after add_jobs task is finished

2021-07-11 08:08:56,358 : INFO : Starting 1 workers in process mode
2021-07-11 08:08:56,369 : INFO : Intiailizing Worker - Worker0 ProcessID - 115
2021-07-11 08:08:56,377 : INFO : Starting process for JobID - 0 on worker - Worker0
